# Generate representative data of vital kits
For senior kits and home essential kits

In [241]:
# get autocomplete working
%config Completer.use_jedi = False

from abc import ABC, abstractmethod
from collections import namedtuple
from enum import Enum

import arrow
import numpy as np

In [245]:

class Temperature(Enum):
    """
    Enum for representing if a temperature is too high, normal, or to low
    """
    high = 0
    normal = 1
    low = 2

TEMPERATURE_RANGES =\
    [(37.2, 42),     # high temperature range
     (34.7, 37.2),   # normal temperature range
     (32, 34.7)]     # low temperature range
"""
List of the ranges of temperature considered high, normal and low respectively
"""

TEMPERATURE_PROBS = [0.05, 0.9, 0.05]
"""
Default probabilities of temperature being high, normal and low respectively
"""

STANDARD_PULSE_OXIMETER_TIME = 5
"""
The default number of secodns that a user users the pulse oximeter for
"""

STANDARD_HEART_RANGE = (50, 90)
"""
The default heart rate for a user
"""

STANDARD_SPO2_RANGE = (60, 100)
"""
Defautl range of SpO2 for a user
"""

STANDARD_SPO2_HEALTH_FACTOR = 10
"""
The default helath weighting for SpO2, the highest this value is the more likely
that SpO2 will be higher
"""

HEART_RATE_STD = 5
"""
The standard deviation when sampling heart rates
"""

SPO2_STD = 3
"""
The standard deviation when sampling SpO2
"""

STANDARD_SYSTOLIC_BP_RANGE = (60, 100)
"""
The standard range of systolic blood pressures
"""

STANDARD_DIASTOLIC_BP_RANGE = (110, 160)
"""
The standard range of diastolic blood pressures
"""

BLOOD_PRESSURE_STD = 5
"""
Standard deviation of blood pressure readings
"""

TODAY = arrow.get('2021-03-02')
"""
The date of "today", all time series will end before this date
"""

BOUGHT_TIME_RANGE = (arrow.get('2019-03-02'), TODAY)
"""
The range of dates users can buy a kit on
"""

STANDARD_USAGE_INTERVAL = 8
"""
The standard mean time between kit uses
"""

THERMOMETER_STR = "thermometer"
PULSE_OXIMITER_STR = "pulse_oximeter"
BLOOD_PRESSURE_CUFF_STR = "blood_pressure"
TEMPERATURE_STR = "temperature"
SYSTOLIC_BP_STR = "systolic_blood_pressure"
DIASTOLIC_BP_STR = "diastolic_blood_pressure"
HEART_RATE_STR = "heart_rate"
SPECIFIC_OXYGEN_STR = "specific_oxygen"

ThermometerReading = namedtuple("Thermometer", ["temperature"])
PulseOximeterReading = namedtuple("PulseOximeterReading", ["spo2", "heart_rate"])
BloodPressureCuffReading = namedtuple("BloodPressureCuffReading", ["systolic_blood_pressure", "diastolic_blood_pressure", "heart_rate"])
    
def _use_thermometer(event_probs=None):
    """
    Using a thermometer is modelled as a hierarchical distribution. Firt we decide if
    the temperature is high, normal or low, and depending on that decision we decide
    a respective temperature
    :param event_probs: List of length 3 that details the probability of temperature
    being [high, normal, low], if not given assumed probabilities are equal
    """
    if not event_probs:
        event_probs = TEMPERATURE_PROBS
    if not len(event_probs) == 3:
        raise ValueError("event_prpbs must be a list of probabilities for [high, medium, low]")
    # make sure event probabilities sum to one
    event_probs /= np.sum(event_probs)
    
    # sample categorical distribution for type of temperature to get the temperature range
    temperature_type = np.random.choice(Temperature, p=event_probs)
    temperature_range = TEMPERATURE_RANGES[temperature_type.value]
    
    # sample a temperate for the type of temperature
    return ThermometerReading(np.random.uniform(*temperature_range))

def _noisy_uniform_sample(base_range, std, num_samples):
    """
    Provides specified number of noisy readings of a value sampled from base range,
    :param base_range: 2-tuple defining range base reading is sampled from
    :param std: the standard deviation of the noise added to reading
    :param num_samples: the number of samples taken
    """
    base_value = np.random.uniform(*base_range)
    
    return np.random.normal(base_value, std, num_samples)
    

def _use_pulse_oximeter(*,
                        average_use_time=None,
                        base_heart_rate_range=None,
                        base_spo2_range=None,
                        spo2_health_factor=None):
    """
    Using a thermometer is modelled as a hierarchical distribution. First we decide how
    long a user uses the device for, then we decide what the base heart rate and SpO2 readings
    are going to be, then we generate noisy readings of these base values
    :param average_use_time: the average number of seconds that a user uses the pulse oximeter
    :param base_heart_range: tuple of highest and lowest heart rate for user
    :param base_spo2_range: tuple of highest and lowest SpO2 for user
    :param spo2_health_factor: higher this number is the more likely they are to have a health SpO2 reading
    :returns: PulseOximeterReading of the readings
    """
    if not average_use_time:
        average_use_time = STANDARD_PULSE_OXIMETER_TIME
    if not base_heart_rate_range:
        base_heart_rate_range = STANDARD_HEART_RANGE
    if not base_spo2_range:
        base_spo2_range = STANDARD_SPO2_RANGE
    if not spo2_health_factor:
        spo2_health_factor = STANDARD_SPO2_HEALTH_FACTOR
    
    # get number of seconds device is used for
    num_secs = np.random.geometric(1/average_use_time)
    
    # get base specific oxygen level
    spo2_low, spo2_high = base_spo2_range
    spo2_weight = np.random.uniform() ** (1 / spo2_health_factor)
    base_spo2 = (spo2_weight * spo2_high - spo2_low) + spo2_low
    
    heart_rates = _noisy_uniform_sample(base_heart_rate_range, HEART_RATE_STD, num_secs)
    spo2s = np.random.normal(base_spo2, SPO2_STD, num_secs)
    # make sure we don't report and SpO2 value of over 100
    spo2s = np.minimum(spo2s, 100)
    
    return PulseOximeterReading(heart_rates, spo2s)

def _use_blood_pressure_monitor(*, systolic_bp_range=None, diastolic_bp_range=None, heart_rate_range=None):
    """
    Using a blood pressure monitor is modelled as taking a noisy uniform sample of systolic, diastolic and heart rate.
    :param systolic_bp_range: range that systolic blood pressure will be sampled from
    :param diastolic_bo_range: range that diastolic blood pressure will be sampled from
    :pram heart_rate_range: range that heart rate will be sampled from
    :returns: tuple of (systolic blood pressure, diastolic blood pressure, heart rate)
    """
    if not systolic_bp_range:
        systolic_bp_range = STANDARD_SYSTOLIC_BP_RANGE
    if not diastolic_bp_range:
        diastolic_bp_range = STANDARD_DIASTOLIC_BP_RANGE
    if not heart_rate_range:
        heart_rate_range = STANDARD_HEART_RANGE
        
    systolic_bp = _noisy_uniform_sample(systolic_bp_range, BLOOD_PRESSURE_STD, 1)
    diastolic_bp = _noisy_uniform_sample(diastolic_bp_range, BLOOD_PRESSURE_STD, 1)
    heart_rate = _noisy_uniform_sample(heart_rate_range, HEART_RATE_STD, 1)
    
    return BloodPressureCuffReading(systolic_bp[0], diastolic_bp[0], heart_rate[0])

class VitalKitUser:
    """
    Abstract base class for all users of a kit.
    
    Presents the `use_kit` method, which randomly selects a kit to use and "uses" it
    """
    
    def __init__(self, kits, mean_time_between_uses=None, bought_time=None):
        if not mean_time_between_uses:
            mean_time_between_uses = STANDARD_USAGE_INTERVAL
        if not bought_time:
            bought_time = _random_bought_time()

        self._kits = kits
        self._mean_time_between_uses = mean_time_between_uses
        self._bought_time = bought_time
    
    def use_kit(self):
        """
        This will select a kit, and use it at a random time after the last use of a kit.
        When the kit is used it returns a tuple of kit name and a list of equal length (possibly singular) time
        series representing the observations from the kit. Time series are of the format
        Tuple[arrow.Arrow, float]
        """
        kit_name = np.random.choice(list(self._kits))
        kit_values = self._kits[kit_name]()
        return kit_name, kit_values
    
    def generate_time_series(self):
        pass

def _random_bought_time():
    """
    Generate a random time in the range BOUGHT_TIME_RANGE
    """
    earliest, latest = BOUGHT_TIME_RANGE
    
    return earliest + np.random.uniform() * (latest - earliest)

class EssentialsKitUser(VitalKitUser):
    """
    The user of a home essentials kit
    """
    KITS = []_use_thermometer,
            PULSE_OXIMITER_STR: _use_pulse_oximeter}
    def __init__(self, mean_time_between_uses=None, bought_time=None):
        super().__init__(self.KITS, mean_time_between_uses, bought_time)

class SeniorKitUser(VitalKitUser):
    """
    The user of a senior kit.
    """
    KITS = {THERMOMETER_STR: _use_thermometer,
            PULSE_OXIMITER_STR: _use_pulse_oximeter,
            BLOOD_PRESSURE_CUFF_STR: _use_blood_pressure_monitor}
    
    def __init__(self, mean_time_between_uses=None, bought_time=None):
        super().__init__(self.KITS, mean_time_between_uses, bought_time)

In [246]:
e = EssentialsKitUser()

In [247]:
e.use_kit()

('thermometer', Thermometer(temperature=36.17992421406855))

In [237]:
s = SeniorKitUser()

In [240]:
s.use_kit()

('thermometer', ('temperature', 35.31408337891727))

In [223]:
dict([a])

{'pulse_oximeter': (array([89.20339836]), array([91.36439541]))}